In [1]:
import os
import numpy as np
from scipy.optimize import minimize
from simsopt.geo import SurfaceRZFourier
from simsopt.objectives import SquaredFlux
from simsopt.objectives import QuadraticPenalty
from simsopt.geo import curves_to_vtk
from simsopt.field import BiotSavart, Current, coils_via_symmetries, Coil
from simsopt.geo import (
    CurveLength, CurveCurveDistance,
    MeanSquaredCurvature, LpCurveCurvature, CurveCWSFourier, ArclengthVariation, CurveXYZFourier
)

import matplotlib.pyplot as plt

In [4]:
wout = 'input.axiTorus_nfp3_QA_final'

ncoils = 1
order = 1 # order of dofs of cws curves
quadpoints = 300 #13 * order
ntheta = 50
nphi = 42

In [5]:
phi = np.linspace(0, 1, 256, endpoint=True)
theta = np.linspace(0, 1, 256, endpoint=True)

In [6]:
# CREATE COIL WINDING SURFACE SURFACE
cws = SurfaceRZFourier.from_vmec_input(wout, range="half period", quadpoints_theta = theta, quadpoints_phi = phi)
cws_full = SurfaceRZFourier.from_vmec_input(wout, range="full torus", quadpoints_theta = theta, quadpoints_phi = phi)

# Poloidally closed curve
poloidal_curve = CurveCWSFourier(mpol=cws.mpol, ntor=cws.ntor, idofs=cws.x,
                                quadpoints=quadpoints, order=order, nfp=cws.nfp, stellsym=cws.stellsym)
poloidal_curve.set_dofs([1, 0, 0, 0, 0, 0, 0, 0])

# Window pane curve
saddle_curve = CurveCWSFourier(mpol=cws.mpol, ntor=cws.ntor, idofs=cws.x,
                                quadpoints=quadpoints, order=order, nfp=cws.nfp, stellsym=cws.stellsym)
saddle_curve.set_dofs([0, np.pi/2, 0.7, 0, 0, np.pi, 0, 0.5])

# Toroidally closed curve
toroidal_curve = CurveCWSFourier(mpol=cws.mpol, ntor=cws.ntor, idofs=cws.x,
                                quadpoints=quadpoints, order=order, nfp=cws.nfp, stellsym=cws.stellsym)
toroidal_curve.set_dofs([0, 0, 0, 0, 1, 0, 0, 0])

In [7]:
OUT_DIR = "./PaperCoils/"
os.makedirs(OUT_DIR, exist_ok=True)

# Save initial curves
curves_to_vtk([poloidal_curve], OUT_DIR + "poloidal_curve", close=True)
curves_to_vtk([toroidal_curve], OUT_DIR + "toroidal_curve", close=True)
curves_to_vtk([saddle_curve], OUT_DIR + "saddle_curve", close=True)
cws_full.to_vtk(OUT_DIR + "cws_init")
